In [3]:
%pip install datasets transformers torch requests pandas scikit-learn tqdm -q

In [ ]:
import os

HF_TOKEN = os.getenv("HF_TOKEN")

#DATASET_NAME = "ArthurMrv/EDGAR-CORPUS-Financial-Summarization-Labeled"

In [4]:
# Authenticate with Hugging Face Hub
from huggingface_hub import login
import getpass

login(token=HF_TOKEN)

print("\nSuccessfully logged in to Hugging Face Hub!")



Successfully logged in to Hugging Face Hub!


In [5]:
from datasets import load_dataset
import pandas as pd

# 1. Load in streaming mode
ds = load_dataset(DATASET_NAME)

df = ds["train"].to_pandas()
df.head()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

EDGAR-CORPUS-Financial-Summarization.csv:   0%|          | 0.00/794M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

,input,summary,model
0,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...,Here's a summary of the financial statement:\n...,Claude
1,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA. ...,"Based on the provided excerpt, here's a summar...",Claude
2,. Report of Independent Registered Public Acco...,This appears to be a partial financial stateme...,Claude
3,Index to Consolidated Financial Statements All...,This appears to be a partial financial stateme...,Claude
4,. ACCOUNTING FIRM To the Board of Directors a...,Here's a summary of the financial statement:\n...,Claude


In [6]:
import hashlib

def hash_row_input(row):
    # Convert row's content to string and hash it
    input_str = str(row["input"])
    # Use SHA256 to hash the input string and return the hex digest
    return hashlib.sha256(input_str.encode("utf-8")).hexdigest()

df["input_hash"] = df.apply(hash_row_input, axis=1)
df.set_index("input_hash", inplace=True)
df.head()

,input,summary,model
input_hash,,,
5e86b75638298802943dcc20093fae925be0c5146cbbf551a839cf47987d1e53,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...,Here's a summary of the financial statement:\n...,Claude
c5b4129b8c9c94491f949c87ac452f30af0f11b8a68e5dc3200a6d3525e18f84,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA. ...,"Based on the provided excerpt, here's a summar...",Claude
a488c022380bcf59d4d3c99127ed2554fa67f86854119e57b9f383a75ab2afa6,. Report of Independent Registered Public Acco...,This appears to be a partial financial stateme...,Claude
52db0b9f1bfe2db5807d09128d511e2176c0226e0558c7dcf60d98fc45efa6bb,Index to Consolidated Financial Statements All...,This appears to be a partial financial stateme...,Claude
a5d562bb3069324a9facf4ecb6fdf52aee051460a2cecfa050314a89555b7f9f,. ACCOUNTING FIRM To the Board of Directors a...,Here's a summary of the financial statement:\n...,Claude


In [15]:
df["llm_sentiment_rationale"] = ""
df["llm_sentiment_class"] = ""
df["llm_sentiment_model"] = ""
df.head()

,input,summary,model,llm_sentiment_class,llm_sentiment_model,llm_sentiment_rationale
input_hash,,,,,,
5e86b75638298802943dcc20093fae925be0c5146cbbf551a839cf47987d1e53,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...,Here's a summary of the financial statement:\n...,Claude,,,
c5b4129b8c9c94491f949c87ac452f30af0f11b8a68e5dc3200a6d3525e18f84,FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA. ...,"Based on the provided excerpt, here's a summar...",Claude,,,
a488c022380bcf59d4d3c99127ed2554fa67f86854119e57b9f383a75ab2afa6,. Report of Independent Registered Public Acco...,This appears to be a partial financial stateme...,Claude,,,
52db0b9f1bfe2db5807d09128d511e2176c0226e0558c7dcf60d98fc45efa6bb,Index to Consolidated Financial Statements All...,This appears to be a partial financial stateme...,Claude,,,
a5d562bb3069324a9facf4ecb6fdf52aee051460a2cecfa050314a89555b7f9f,. ACCOUNTING FIRM To the Board of Directors a...,Here's a summary of the financial statement:\n...,Claude,,,


In [16]:
df_for_upload = df.copy()

print(f"DataFrame shape: {df_for_upload.shape}")
print(f"Columns: {df_for_upload.columns.tolist()}")
print(f"\nSample data:")
print(df_for_upload.head(2))

DataFrame shape: (10610, 6)
Columns: ['input', 'summary', 'model', 'llm_sentiment_class', 'llm_sentiment_model', 'llm_sentiment_rationale']

Sample data:
                                                                                                input  \
input_hash                                                                                              
5e86b75638298802943dcc20093fae925be0c5146cbbf55...  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...   
c5b4129b8c9c94491f949c87ac452f30af0f11b8a68e5dc...  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA.  ...   

                                                                                              summary  \
input_hash                                                                                              
5e86b75638298802943dcc20093fae925be0c5146cbbf55...  Here's a summary of the financial statement:\n...   
c5b4129b8c9c94491f949c87ac452f30af0f11b8a68e5dc...  Based on the provided excerpt, here's a summar...   

    

In [17]:
# Convert DataFrame to Hugging Face Dataset and upload
from datasets import Dataset

# Reset index to make it a regular column if needed
if df_for_upload.index.name is not None:
    df_for_upload = df_for_upload.reset_index()

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_for_upload)

# Display dataset info
print(f"Dataset shape: {len(dataset)} rows")
print(f"Dataset features: {dataset.features}")
print(f"\nFirst few examples:")
print(dataset[:3])

Dataset shape: 10610 rows
Dataset features: {'input_hash': Value('string'), 'input': Value('string'), 'summary': Value('string'), 'model': Value('string'), 'llm_sentiment_class': Value('string'), 'llm_sentiment_model': Value('string'), 'llm_sentiment_rationale': Value('string')}

First few examples:
{'input_hash': ['5e86b75638298802943dcc20093fae925be0c5146cbbf551a839cf47987d1e53', 'c5b4129b8c9c94491f949c87ac452f30af0f11b8a68e5dc3200a6d3525e18f84', 'a488c022380bcf59d4d3c99127ed2554fa67f86854119e57b9f383a75ab2afa6'], 'input': ['FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA INDEX TO CONSOLIDATED FINANCIAL STATEMENTS  ACCOUNTING FIRM To the Board of Directors and Stockholders Digerati Technologies, Inc. San Antonio, Texas We have audited the accompanying consolidated balance sheets of Digerati Technologies, Inc. and its subsidiaries (collectively, “Digerati”) as of July 31, 2016 and July 31, 2015, and the related consolidated statements of operations, stockholders’ deficit and cash flows fo

In [18]:
# Upload to Hugging Face Hub
# Replace 'your-username' with your actual Hugging Face username
# You can also make it private by adding private=True

print(f"\nUploading dataset to {DATASET_NAME}...")
try:
    dataset.push_to_hub(
        repo_id=DATASET_NAME,
        token=HF_TOKEN,
        private=False  # Set to True if you want a private dataset
    )
    print(f"\n✅ Successfully uploaded dataset to: https://huggingface.co/datasets/{DATASET_NAME}")
except Exception as e:
    print(f"\n❌ Error uploading dataset: {e}")
    print("Make sure:")
    print("1. The repository name is in the format 'username/dataset-name'")
    print("2. You have write access to the repository")
    print("3. The repository exists or you have permission to create it")



Uploading dataset to ArthurMrv/EDGAR-CORPUS-Financial-Summarization-Labeled...


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :  19%|#8        | 33.4MB /  176MB            

README.md: 0.00B [00:00, ?B/s]


✅ Successfully uploaded dataset to: https://huggingface.co/datasets/ArthurMrv/EDGAR-CORPUS-Financial-Summarization-Labeled
